In [0]:
!pip install fuzzywuzzy
!pip install langdetect
!pip install stop_words
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import urllib.request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from fuzzywuzzy import fuzz
from string import digits
from stop_words import get_stop_words
from nltk.corpus import stopwords
from fuzzywuzzy import process

     |████████████████████████████████| 983kB 2.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=5375a79c5c0c8c552b3869c22d3a6ee817c1e3a2d5b70af26dce942251a9e9fa
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=9f6ee7319e4545db367073710ae7923e59f9da43e4a272ec8b5b30d7eb71c30e
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
punct = punctuation+'©«»—…“”*№– ' + digits
stop_words_list = get_stop_words('en')
stop_words_list2 = stopwords.words('english')
newlist = stop_words_list + stop_words_list2
for word in ["свой"]:
    newlist.append(word)
newlist_set = set(newlist)

In [0]:
def which_language (text):
  import re
  if bool(re.search('[\u0400-\u04FF]', text)) == True:
    return 'russian'
  else:
    return 'english'

In [0]:
teams_names_in_russian_and_english_df = pd.read_csv("teams names in russian and english.csv")
leagues_names_in_russian_and_english_df = pd.read_csv("leagues names in russian and english.csv")

In [0]:
df = pd.read_csv('result.tar.gz', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df['result.tsv'] = df['result.tsv'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'https\S+', '', regex=True)
df = df.replace('', np.NaN)
df = df.replace('\xa0', np.NaN)
df = df.dropna()
df =  df[df['result.tsv'].str.contains('[A-Za-z]')]
df = df[df['result.tsv'].str.split().str.len().gt(1)]

In [0]:
df = df.dropna()

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.reset_index(drop=True)

In [0]:
df.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
1,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония
2,Украинская команда имеет отличный шанс выйти в...,1.55,Заря забьет 1 или 2 гола: да,1xСтавка,2019-08-15 17:00:00,Лига Европы УЕФА,Заря,ЦСКА София
3,«Сент-Миррен» в родных стенах встретится с «Аб...,2.55,«Абердин» фора (-1),BETCITY,2019-08-11 14:00:00,Премьершип,Сент-Миррен,Абердин
4,На стадионе «Айброкс» состоится встреча между ...,1.97,"«Абердин» фора (+1,5)",Марафон,2019-09-28 14:00:00,Премьершип,Глазго Рейнджерс,Абердин


In [0]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
  lang.append(which_language(str(text)))

In [0]:
df['Language'] = lang

In [0]:
grouped=df.groupby('Language')

In [0]:
for name, group in grouped: 
  if name == 'english': 
    english = group.drop('Language',axis=1) 
  else: 
    russian = group.drop('Language',axis=1)

In [0]:
english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_1_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_2_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, leagues_names_in_russian_and_english_df,  how='left', left_on=['league_name_in_russian'], right_on = ['leagues name in Russian'])
english.rename({'unique_leagues_in_english': 'league_name_in_english'}, axis=1, inplace=True)


In [0]:
english.drop(['team name in Russian_x' , 'team name in Russian_y' , 'leagues name in Russian' ] , axis=1 , inplace=True)

In [0]:
# Here we are just re-naming the columns so one check that what's in it and interpret the information easily 
english.rename({'team name in English_x': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)
english.rename({'team name in English_y': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)
english.rename({'leagues names in English': 'league_name_in_english'}, axis=1, inplace=True)

In [0]:
# Here we are just setting the correct data type that english names should be string
english['team_of_season_1_name_in_english']=english['team_of_season_1_name_in_english'].astype(str)
english['team_of_season_2_name_in_english']=english['team_of_season_2_name_in_english'].astype(str)
english['league_name_in_english']=english['league_name_in_english'].astype(str)

In [0]:
english.head(2)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League


In [0]:
unique_leagues = english['league_name_in_english'].unique().tolist()
len(unique_leagues)

78

In [0]:
unique_teams1 = english['team_of_season_1_name_in_english'].unique().tolist()
len(unique_teams1)

1233

In [0]:
unique_teams2 = english['team_of_season_2_name_in_english'].unique().tolist()
len(unique_teams2)

1126

In [0]:
unique_teams = unique_teams1 + unique_teams2
unique_teams_set = set(unique_teams)
len(unique_teams_set)

1233

In [0]:
english = pd.read_csv('english_custom_lists_and_directmatch.csv')

In [0]:
english = english[english['result.tsv'].map(len) > 30]

In [0]:
english = english.reset_index(drop=True)

In [0]:
english.shape

(87756, 13)

In [0]:
english.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Middlesbrough have woken to good times in thei...,7.0,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship,0.0,0.0
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League,0.0,0.0
2,In my opinion a 0-0 draw is quite likely betwe...,13.0,0-0,bet365,2019-04-18 19:00:00,Лига Европы УЕФА,Валенсия,Вильярреал,Valencia,Villarreal,UEFA Europa League,0.0,0.0


In [0]:
subset_english = english.sample(n=10000)
subset_english.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
2928,Burnley will not lose Manchester City. The las...,5.65,1X,bet365,2019-04-28 13:05:00,Премьер-лига,Бернли,Манчестер Сити,Burnley,Manchester city,Premier League,0.0,0.0
21674,Expecting derry city will win this match and s...,1.86,H1 (-2.25),bet365,2019-07-12 18:45:00,Премьер-лига,Дерри Сити,ЮСД,Derry City,YUSD,Premier League,0.0,0.0
14599,Scored against Germany In a recent friendly in...,6.50,First Goalscorer: Dominic Solanke,bet365,2019-06-21 16:30:00,Чемпионат Европы U21,Англия U21,Румыния U21,England U21,Romania U21,U21 European Championship,0.0,0.0
62063,Esbjerg & Yes appeals today as they have pulle...,5.50,W1 & BTTS (yes),bet365,2019-04-18 14:00:00,Суперлига,Эсбьерг ФБ,Брондбю,Esbjerg FB,Brondby,Super,0.0,0.0
86633,The match can be draw because both teams lost ...,4.50,X & BTTS (yes),bet365,2019-09-22 15:30:00,Премьер-лига,Челси,Ливерпуль,chelsea,Liverpool,Premier League,0.0,0.0


In [0]:
subset_english = subset_english.reset_index(drop=True)
subset_english.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Burnley will not lose Manchester City. The las...,5.65,1X,bet365,2019-04-28 13:05:00,Премьер-лига,Бернли,Манчестер Сити,Burnley,Manchester city,Premier League,0.0,0.0
1,Expecting derry city will win this match and s...,1.86,H1 (-2.25),bet365,2019-07-12 18:45:00,Премьер-лига,Дерри Сити,ЮСД,Derry City,YUSD,Premier League,0.0,0.0
2,Scored against Germany In a recent friendly in...,6.50,First Goalscorer: Dominic Solanke,bet365,2019-06-21 16:30:00,Чемпионат Европы U21,Англия U21,Румыния U21,England U21,Romania U21,U21 European Championship,0.0,0.0
3,Esbjerg & Yes appeals today as they have pulle...,5.50,W1 & BTTS (yes),bet365,2019-04-18 14:00:00,Суперлига,Эсбьерг ФБ,Брондбю,Esbjerg FB,Brondby,Super,0.0,0.0
4,The match can be draw because both teams lost ...,4.50,X & BTTS (yes),bet365,2019-09-22 15:30:00,Премьер-лига,Челси,Ливерпуль,chelsea,Liverpool,Premier League,0.0,0.0


Team 1:

In [0]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85:
                    #print(english['team_of_season_1_name_in_english'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams1'][i] = 1

                    break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10000it [24:42,  6.74it/s]


In [0]:
success = subset_english[subset_english['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

6492
10000
0.6492


In [0]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
#         print(i, good_words)
#         print(subset_english['team_of_season_1_name_in_english'][i])
        
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 65 and fuzz.ratio(word, subset_english['team_of_season_1_name_in_english'][i]) > 65:
#                     print(subset_english['team_of_season_1_name_in_english'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams1'][i] = 1

                    break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10000it [07:13, 23.07it/s]


In [0]:
success = subset_english[subset_english['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

7401
10000
0.7401


Team2:

In [0]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85 and subset_english['team_of_season_2_name_in_english'][i] == word:
                    #print(english['team_of_season_2_name_in_english'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams2'][i] = 1

                    break


3it [00:00,  3.22it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10000it [31:32,  5.28it/s]


In [0]:
success = subset_english[subset_english['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

3396
10000
0.3396


In [0]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        

        for word in good_words:
            for team in unique_teams:
                if (pd.isnull(subset_english['team_of_season_2_name_in_english'][i])==False):
                  if fuzz.ratio(word, team) > 35 and fuzz.ratio(word, subset_english['team_of_season_2_name_in_english'][i]) > 35:                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                      subset_english['list_direct_match_for_teams2'][i] = 1

                      break

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10000it [31:51,  5.23it/s]


In [0]:
success = subset_english[subset_english['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

7171
10000
0.7171


In [0]:
subset_english.to_csv('fuzzylistsnew20106.csv', index = False)

League:

In [0]:
leagues_match = np.zeros(subset_english.shape[0])

In [0]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    #print(i)
    capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
    quotes = re.findall(r'\"(.+?)\"', text)
    quotes2 = re.findall(r'«(.+?)»', text)
    quotes3 = re.findall(r'\'(.+?)\'', text)
    search_words = capital_words_en + quotes + quotes2 + quotes3
    search_words = set(search_words)
    good_words = [word for word in search_words if word.lower() not in newlist_set] 
    #print(good_words)
    
    for word in good_words:
        for league in unique_leagues:
            if fuzz.ratio(word, league) > 50 and fuzz.ratio(word, subset_english['league_name_in_english'][i]) > 50:
                print(subset_english['league_name_in_english'][i], word, league, fuzz.ratio(word, league))
                #print('yes')
                leagues_match[i] = 1

                break

24it [00:00, 114.78it/s]

example Camp example 55
Premier League League Premier League 60


54it [00:00, 121.11it/s]

League One League Premier League 60


112it [00:00, 125.13it/s]

...

Premier League Premier Premier League 67
Premier League League Premier League 60
Championship Championship Championship 100


9982it [01:16, 107.87it/s]

Premier League Premiership Championship 52
Primeyra League Primeira Premier League 55
Primeyra League Primera Premier League 57
gold Cup Cup gold Cup 55


10000it [01:16, 131.21it/s]

gold Cup Cup gold Cup 55
UEFA Europa League European UEFA Europa League 54


In [0]:
subset_english['list_fuzzy_match_for_league'] = leagues_match

In [0]:
success = subset_english[subset_english['list_fuzzy_match_for_league'] == 1].shape[0]
print(success)
all_vals = subset_english['list_fuzzy_match_for_league'].shape[0]
print(all_vals)
baseline_league = success / all_vals
print(baseline_league)

827
10000
0.0827


In [0]:
subset_english.to_csv('fuzzylists0306_withleagues.csv', index = False)